In [21]:
from pathlib import Path
import librosa
import torch
from argparse import ArgumentParser
import matplotlib
import h5py
import tqdm
from IPython.display import Audio, display

import sys
sys.path += ['../src']

import utils
import wavenet_models
from utils import save_audio
from wavenet import WaveNet
from wavenet_generator import WavenetGenerator
from nv_wavenet_generator import NVWavenetGenerator
from nv_wavenet_generator import Impl

In [22]:
data = Path.cwd().parent.joinpath("musicnet", "parsed")

data = {label.stem : [wav for wav in data.joinpath(label).iterdir()] for label in data.iterdir()}

for key, value in data.items():
    print(key)
    print(value[0].stem)

Beethoven_Accompanied_Violin
2330
Bach_Solo_Cello
2217
Bach_Solo_Piano
2194
Beethoven_Solo_Piano
2322
Beethoven_String_Quartet
2313
Cambini_Wind_Quintet
2075


In [23]:
checkpoint = Path('../checkpoints/pretrained_musicnet/bestmodel_0.pth')

model_args = torch.load(checkpoint.parent / 'args.pth')[0]
encoder = wavenet_models.Encoder(model_args)
encoder.load_state_dict(torch.load(checkpoint)['encoder_state'])
encoder.eval()
encoder = encoder.cuda()

In [24]:
processed_data = {}
file_names = {}
for label in data:
    processed_data[label] = []
    file_names[label] = []
    for path in data[label]:
        processed, rate = librosa.load(path, sr=16000, duration=10)
        assert rate == 16000
        processed = utils.mu_law(processed)
        processed_data[label].append(torch.tensor(processed).unsqueeze(0).float().cuda())
        file_names[label].append(path.stem)

In [26]:
for label in file_names:
    print(label)
    for file_name in file_names[label]:
        print(file_name)

Beethoven_Accompanied_Violin
2330
2334
2335
2336
2341
2342
2397
2398
2420
2462
2463
2466
2501
2502
2570
2571
2572
2573
2626
2627
2628
2629
Bach_Solo_Cello
2217
2218
2219
2220
2221
2222
2293
2294
2295
2296
2297
2298
Bach_Solo_Piano
2194
2195
2196
2198
2200
2201
2207
2208
2209
2210
2211
2212
2213
2214
2215
2224
2225
2227
2228
2229
2230
2231
2232
2234
2237
2238
2239
2240
2247
2248
2292
2300
2302
2303
2304
2305
2307
2308
2310
Beethoven_Solo_Piano
2322
2325
2343
2345
2346
2348
2350
2357
2358
2359
2364
2371
2372
2373
2374
2388
2389
2390
2391
2392
2393
2404
2405
2406
2410
2411
2422
2423
2424
2436
2441
2442
2443
2444
2471
2472
2473
2476
2477
2478
2486
2487
2488
2490
2491
2492
2509
2510
2512
2514
2516
2527
2528
2529
2530
2531
2532
2533
2537
2538
2540
2542
2550
2555
2556
2557
2564
2566
2567
2568
2575
2576
2581
2582
2588
2590
2591
2593
2594
2595
2596
2603
2607
2608
2611
2614
2618
2619
2620
2632
2633
2677
2678
Beethoven_String_Quartet
2313
2314
2315
2365
2366
2368
2376
2377
2379
2381
2382
2383
238

In [28]:
with torch.no_grad():
    for label in processed_data:
        for sample, file_name in zip(processed_data[label], file_names[label]):
            torch.save(encoder(sample), "/datasets/musicnet/musicnet/encoded/" + label + "/" + file_name + ".pt")